In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [12]:
df_demand = pd.read_csv('Data/demand.csv')
df_vehicles = pd.read_csv('Data/vehicles.csv')
df_vehicles_fuels = pd.read_csv('Data/vehicles_fuels.csv')
df_fuels = pd.read_csv('Data/fuels.csv')
df_carbon_emissions = pd.read_csv('Data/carbon_emissions.csv')
df_cost = pd.read_csv('Data/cost_profiles.csv')

In [13]:
display(df_vehicles)

,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance
0,BEV_S1_2023,BEV,S1,2023,187000,102000,D1
1,BEV_S1_2024,BEV,S1,2024,177650,102000,D1
2,BEV_S1_2025,BEV,S1,2025,168767,102000,D1
3,BEV_S1_2026,BEV,S1,2026,160329,102000,D2
4,BEV_S1_2027,BEV,S1,2027,152312,102000,D2
...,...,...,...,...,...,...,...
187,LNG_S3_2034,LNG,S3,2034,209208,73000,D4
188,LNG_S3_2035,LNG,S3,2035,215484,73000,D4
189,LNG_S3_2036,LNG,S3,2036,221948,73000,D4
190,LNG_S3_2037,LNG,S3,2037,228607,73000,D4


In [3]:
df_sample = pd.read_csv('Data/sample_submission.csv')
display(df_sample)

,Year,ID,Num_Vehicles,Type,Fuel,Distance_bucket,Distance_per_vehicle(km)
0,2023,BEV_S1_2023,3,Buy,NaN,NaN,0.0
1,2023,Diesel_S1_2023,36,Buy,NaN,NaN,0.0
2,2023,LNG_S1_2023,34,Buy,NaN,NaN,0.0
3,2023,BEV_S2_2023,4,Buy,NaN,NaN,0.0
4,2023,Diesel_S2_2023,15,Buy,NaN,NaN,0.0
...,...,...,...,...,...,...,...
1682,2038,BEV_S3_2037,4,Use,Electricity,D3,73000.0
1683,2038,BEV_S3_2038,3,Use,Electricity,D3,73000.0
1684,2038,BEV_S3_2033,3,Use,Electricity,D4,73000.0
1685,2038,BEV_S3_2036,1,Use,Electricity,D4,73000.0


In [4]:
def calculate_purchase_cost(submission_row, vehicles):
    """
    Calculate the cost for buying vehicles (one row)
    Used when Type == Buy
    """

    if submission_row['Type'] == 'Buy': #type check
        vehicle_id = submission_row['ID']
        num_vehicles = submission_row['Num_Vehicles']
        purchase_cost = vehicles.set_index('ID').loc[vehicle_id, 'Cost ($)'] * num_vehicles
        return purchase_cost
    return 0 #if we don't buy, return 0
#print(df_sample.iloc[1])
#print(calculate_purchase_cost(df_sample.iloc[1], df_vehicles))

In [5]:
def calculate_insurance_cost(submission_row, vehicles, cost_profiles):
    """
    Calculate the cost for insurance
    Used when Type == Use
    """

    if submission_row['Type'] == 'Use':
        vehicle_id = submission_row['ID']
        num_vehicles = submission_row['Num_Vehicles']
        year_purchased = int(vehicle_id.split('_')[-1])
        years_used = submission_row['Year'] - year_purchased
        if years_used == 0:
            return 0
        elif years_used < 10:
            insurance_rate = cost_profiles.set_index('End of Year').loc[years_used, 'Insurance Cost %'] / 100
            insurance_cost = insurance_rate * vehicles.set_index('ID').loc[vehicle_id, 'Cost ($)'] * num_vehicles
            return insurance_cost
    return 0

#print(df_sample.iloc[142])
#print(calculate_insurance_cost(df_sample.iloc[142], df_vehicles, df_cost))

In [6]:
def calculate_maintenance_cost(submission_row, vehicles, cost_profiles):
    """
    Calculate the cost for maintenance
    Used when Type == Use
    """

    if submission_row['Type'] == 'Use':
        vehicle_id = submission_row['ID']
        num_vehicles = submission_row['Num_Vehicles']
        year_purchased = int(vehicle_id.split('_')[-1])
        years_used = submission_row['Year'] - year_purchased
        if years_used == 0:
            return 0
        elif years_used < 10:
            maintenance_rate = cost_profiles.set_index('End of Year').loc[years_used, 'Maintenance Cost %'] / 100
            maintenance_cost = maintenance_rate * vehicles.set_index('ID').loc[vehicle_id, 'Cost ($)'] * num_vehicles
            return maintenance_cost
    return 0

#print(df_sample.iloc[468])
#print(calculate_maintenance_cost(df_sample.iloc[468], df_vehicles, df_cost))


In [7]:
def calculate_fuel_cost(submission_row, vehicles_fuels, fuels):
    """
    Calculate the cost for fuel
    Used when Type == Use
    """
    if submission_row['Type'] == 'Use':
        vehicle_id = submission_row['ID']
        num_vehicles = submission_row['Num_Vehicles']
        distance_per_vehicle = submission_row['Distance_per_vehicle(km)']
        fuel_type = submission_row['Fuel']
        year = submission_row['Year']
    
        fuel_consumption = vehicles_fuels[(vehicles_fuels['ID'] == vehicle_id) & (vehicles_fuels['Fuel'] == fuel_type)]['Consumption (unit_fuel/km)'].values[0]
        fuel_cost_per_unit = fuels[(fuels['Fuel'] == fuel_type) & (fuels['Year'] == year)]['Cost ($/unit_fuel)'].values[0]
        
        fuel_cost = fuel_consumption * distance_per_vehicle * num_vehicles * fuel_cost_per_unit
        return fuel_cost
    return 0

#for i in range(df_sample.shape[0]):
#    print(calculate_fuel_cost(df_sample.iloc[i], df_vehicles_fuels, df_fuels))

In [8]:
def calculate_resale_revenue(submission_row, vehicles, cost_profiles):
    """
    Calculate the resale revenue
    Used when Type == Sell
    """

    if submission_row['Type'] == 'Sell':
        vehicle_id = submission_row['ID']
        num_vehicles = submission_row['Num_Vehicles']
        year_purchased = int(vehicle_id.split('_')[-1])
        years_used = submission_row['Year'] - year_purchased
        if years_used == 0:
            return 0
        elif years_used < 10:
            resale_rate = cost_profiles.set_index('End of Year').loc[years_used, 'Resale Value %'] / 100
            resale_revenue = resale_rate * vehicles.set_index('ID').loc[vehicle_id, 'Cost ($)'] * num_vehicles
            return resale_revenue
    return 0

#print(df_sample.iloc[1200])
#print(calculate_resale_revenue(df_sample.iloc[1200], df_vehicles, df_cost))


In [9]:
def calculate_total_cost_for_row(submission_row, vehicles, vehicles_fuels, fuels, cost_profiles):
    total_cost = 0
    total_cost += calculate_purchase_cost(submission_row, vehicles)
    total_cost += calculate_insurance_cost(submission_row, vehicles, cost_profiles)
    total_cost += calculate_maintenance_cost(submission_row, vehicles, cost_profiles)
    total_cost += calculate_fuel_cost(submission_row, vehicles_fuels, fuels)
    total_cost -= calculate_resale_revenue(submission_row, vehicles, cost_profiles)
    return total_cost

In [10]:
def calculate_total_cost(submission, vehicles, vehicles_fuels, fuels, cost_profiles):
    """
    Calculate the total cost
    """
    
    total_cost = 0
    
    for i in range(submission.shape[0]):
        row = submission.iloc[i].copy()
        total_cost += calculate_total_cost_for_row(row, vehicles, vehicles_fuels, fuels, cost_profiles)
    return total_cost

print(calculate_total_cost(df_sample, df_vehicles, df_vehicles_fuels, df_fuels, df_cost))

219411731.00799724


In [75]:
def calculate_carbon_emission(submission_row, vehicles_fuels, fuels):
    """
    Calculate the total carbon emission for a single row based on the vehicle usage.
    """
    if submission_row['Type'] == 'Use':
        vehicle_id = submission_row['ID']
        num_vehicles = submission_row['Num_Vehicles']
        distance_per_vehicle = submission_row['Distance_per_vehicle(km)']
        fuel_type = submission_row['Fuel']
        year = submission_row['Year']
        
        fuel_consumption = vehicles_fuels[(vehicles_fuels['ID'] == vehicle_id) & (vehicles_fuels['Fuel'] == fuel_type)]['Consumption (unit_fuel/km)'].values[0]
        carbon_emission_per_unit = fuels[(fuels['Fuel'] == fuel_type) & (fuels['Year'] == year)]['Emissions (CO2/unit_fuel)'].values[0]

        total_carbon_emission = fuel_consumption * distance_per_vehicle * num_vehicles * carbon_emission_per_unit
        return total_carbon_emission
    return 0

# Example of using the function
for i in range(df_sample.shape[0]):
    print(calculate_carbon_emission(df_sample.iloc[i], df_vehicles_fuels, df_fuels))


0
0
0
0
0
0
0
0
0
0
0
0
0.0
208043.92788767998
125913.34429869599
901523.6875132798
82878.456618126
1178915.59136352
102004.25429923201
33276.873919104
83942.229532464
0.0
217012.97219087995
130639.23962037597
80993.460874816
304824.89244754397
46281.977642752
26826.193467160003
72337.65739695998
6706.548366790001
0.0
316315.61235152
147219.612475248
79078.90308788
7416.792842576001
12648.764683664
0.0
79765.46565264
45495.67924148
847766.69231796
508538.17722248397
55835.825956848
31846.975469036
15953.093130528001
29914.010424851997
0
0
0
0
0
0
0
0
0
0
0
0
0
0
208043.92788767998
125913.34429869599
0.0
6375.2658937020005
693479.7596256
70127.92483072201
251826.68859739197
166384.36959552
503653.37719478394
251826.68859739197
138695.95192511997
83942.229532464
6375.2658937020005
20119.645100370002
0.0
23140.988821376
87092.82641358399
506363.6017787199
217732.06603395997
87092.82641358399
130639.23962037597
289350.62958783994
6706.548366790001
72337.65739695998
6706.548366790001
0.0
31